In [1]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px

from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [3]:
tv = TfidfVectorizer(max_features=5000)
stem = StemmerFactory().create_stemmer()
stopword = StopWordRemoverFactory().create_stop_word_remover()

In [4]:
tourism_rating = pd.read_csv("/content/sample_data/tourism_rating.csv")
tourism_with_id = pd.read_csv("/content/sample_data/tourism_with_id.csv")
users = pd.read_csv("/content/sample_data/user.csv")

In [5]:
tourism_rating.head()

,User_Id,Place_Id,Place_Ratings
0,1,179,3
1,1,344,2
2,1,5,5
3,1,373,3
4,1,101,4


In [6]:
tourism_with_id.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Unnamed: 11,Unnamed: 12
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,NaN,1
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,NaN,2
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,NaN,3
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,106.895156,NaN,4
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134,NaN,5


In [7]:
users.head()

,User_Id,Location,Age
0,1,"Semarang, Jawa Tengah",20
1,2,"Bekasi, Jawa Barat",21
2,3,"Cirebon, Jawa Barat",23
3,4,"Bekasi, Jawa Barat",21
4,5,"Lampung, Sumatera Selatan",20


In [8]:
tourism_rating.isna().sum()

User_Id          0
Place_Id         0
Place_Ratings    0
dtype: int64

In [9]:
tourism_with_id.isna().sum()

Place_Id          0
Place_Name        0
Description       0
Category          0
City              0
Price             0
Rating            0
Time_Minutes    232
Coordinate        0
Lat               0
Long              0
Unnamed: 11     437
Unnamed: 12       0
dtype: int64

In [10]:
users.isna().sum()

User_Id     0
Location    0
Age         0
dtype: int64

In [11]:
tourism_with_id.drop(['Time_Minutes','Unnamed: 11','Unnamed: 12','Coordinate','Lat','Long','Rating'], axis=1 , inplace=True)
tourism_with_id

,Place_Id,Place_Name,Description,Category,City,Price
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000
...,...,...,...,...,...,...
432,433,Museum Mpu Tantular,Museum Negeri Mpu Tantular adalah sebuah museu...,Budaya,Surabaya,2000
433,434,Taman Bungkul,Taman Bungkul adalah taman wisata kota yang te...,Taman Hiburan,Surabaya,0
434,435,Taman Air Mancur Menari Kenjeran,Air mancur menari atau dancing fountain juga a...,Taman Hiburan,Surabaya,0
435,436,Taman Flora Bratang Surabaya,Taman Flora adalah salah satu taman kota di Su...,Taman Hiburan,Surabaya,0


In [12]:
data_df= pd.merge(tourism_rating.groupby('Place_Id')['Place_Ratings'].mean() , tourism_with_id ,on='Place_Id')
data_df

,Place_Id,Place_Ratings,Place_Name,Description,Category,City,Price
0,1,3.722222,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000
1,2,2.840000,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0
2,3,2.526316,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000
3,4,2.857143,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000
4,5,3.520000,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000
...,...,...,...,...,...,...,...
432,433,3.304348,Museum Mpu Tantular,Museum Negeri Mpu Tantular adalah sebuah museu...,Budaya,Surabaya,2000
433,434,2.400000,Taman Bungkul,Taman Bungkul adalah taman wisata kota yang te...,Taman Hiburan,Surabaya,0
434,435,3.000000,Taman Air Mancur Menari Kenjeran,Air mancur menari atau dancing fountain juga a...,Taman Hiburan,Surabaya,0
435,436,3.090909,Taman Flora Bratang Surabaya,Taman Flora adalah salah satu taman kota di Su...,Taman Hiburan,Surabaya,0


In [13]:
def preprocessing(data):
  data = data.lower()
  data = stem.stem(data)
  data=stopword.remove(data)
  return data

In [14]:
data_content_filtering = data_df.copy()
data_content_filtering['Tag'] = data_content_filtering['Description']+' '+data_content_filtering['Category']
data_content_filtering.drop(['Place_Ratings' ,'Description','Category' ,'City','Price'] , axis=1, inplace=True)
data_content_filtering


,Place_Id,Place_Name,Tag
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu..."
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...
...,...,...,...
432,433,Museum Mpu Tantular,Museum Negeri Mpu Tantular adalah sebuah museu...
433,434,Taman Bungkul,Taman Bungkul adalah taman wisata kota yang te...
434,435,Taman Air Mancur Menari Kenjeran,Air mancur menari atau dancing fountain juga a...
435,436,Taman Flora Bratang Surabaya,Taman Flora adalah salah satu taman kota di Su...


In [15]:
data_content_filtering.Tag = data_content_filtering.Tag.apply(preprocessing)
data_content_filtering

,Place_Id,Place_Name,Tag
0,1,Monumen Nasional,monumen nasional populer singkat monas tugu mo...
1,2,Kota Tua,kota tua jakarta juga nama kota tua pusat alun...
2,3,Dunia Fantasi,dunia fantasi sebut dufan tempat hibur letak k...
3,4,Taman Mini Indonesia Indah (TMII),taman mini indonesia indah rupa suatu kawasan ...
4,5,Atlantis Water Adventure,atlantis water adventure kenal atlantis ancol ...
...,...,...,...
432,433,Museum Mpu Tantular,museum negeri mpu tantular buah museum negeri ...
433,434,Taman Bungkul,taman bungkul taman wisata kota letak pusat ko...
434,435,Taman Air Mancur Menari Kenjeran,air mancur tari dancing fountain ada kawasan j...
435,436,Taman Flora Bratang Surabaya,taman flora salah satu taman kota surabaya rup...


In [16]:
vectors =  tv.fit_transform(data_content_filtering.Tag).toarray()
vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
similarity = cosine_similarity(vectors)
similarity[0][1:10]

array([0.04510993, 0.07188361, 0.02807929, 0.00479488, 0.01497397,
       0.02458836, 0.00394491, 0.01975822, 0.01633207])

In [18]:
def recommend_by_content_based_filtering(nama_tempat):
    nama_tempat_index = data_content_filtering[data_content_filtering['Place_Name']==nama_tempat].index[0]
    distancess = similarity[nama_tempat_index]
    nama_tempat_list = sorted(list(enumerate(distancess)),key=lambda x: x[1],reverse=True)[1:10]

    recommended_nama_tempats = []
    for i in nama_tempat_list:
        recommended_nama_tempats.append([data_content_filtering.iloc[i[0]].Place_Name]+[i[1]])


    return recommended_nama_tempats

In [19]:
recommend_by_content_based_filtering('Monumen Nasional')

[['Monumen Bandung Lautan Api', 0.28683907306335177],
 ['Monumen Selamat Datang', 0.2661577967639709],
 ['Monumen Perjuangan Rakyat Jawa Barat', 0.25729692362006246],
 ['Tugu Muda Semarang', 0.22491138935052768],
 ['Monumen Bambu Runcing Surabaya', 0.20482652874129093],
 ['Monumen Tugu Pahlawan', 0.18791275217429565],
 ['Monumen Sanapati', 0.1795335499410377],
 ['Monumen Yogya Kembali', 0.17871073001225599],
 ['Monumen Palagan Ambarawa', 0.16871094841229334]]

In [20]:
data_collaborative = tourism_rating.copy()
data_collaborative.head()

,User_Id,Place_Id,Place_Ratings
0,1,179,3
1,1,344,2
2,1,5,5
3,1,373,3
4,1,101,4


In [21]:

user_ids = data_collaborative['User_Id'].unique().tolist()
print('list userID: ', user_ids)

#  encoding userID
user_to_user_encoded = {x: i for i, x in enumerate(user_ids)}
print('encoded userID : ', user_to_user_encoded)


user_encoded_to_user = {i: x for i, x in enumerate(user_ids)}
print('encoded angka ke userID: ', user_encoded_to_user)

list userID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219

In [22]:

resto_ids = data_collaborative['Place_Id'].unique().tolist()


resto_to_resto_encoded = {x: i for i, x in enumerate(resto_ids)}


resto_encoded_to_resto = {i: x for i, x in enumerate(resto_ids)}

In [23]:
data_collaborative['user'] = data_collaborative['User_Id'].map(user_to_user_encoded)


data_collaborative['resto'] = data_collaborative['Place_Id'].map(resto_to_resto_encoded)

In [24]:
num_users = len(user_to_user_encoded)
print(num_users)


num_resto = len(resto_encoded_to_resto)
print(num_resto)


data_collaborative['Place_Ratings'] = data_collaborative['Place_Ratings'].values.astype(np.float32)

#  minimum rating
min_rating = min(data_collaborative['Place_Ratings'])

# Maximum rating
max_rating = max(data_collaborative['Place_Ratings'])

print('Number of User: {}, Number of Resto: {}, Min Rating: {}, Max Rating: {}'.format(
    num_users, num_resto, min_rating, max_rating
))

300
437
Number of User: 300, Number of Resto: 437, Min Rating: 1.0, Max Rating: 5.0


In [25]:
data_collaborative = data_collaborative.sample(frac=1, random_state=42)
data_collaborative

,User_Id,Place_Id,Place_Ratings,user,resto
6252,188,207,3.0,187,262
4684,142,268,2.0,141,83
1731,54,103,3.0,53,10
4742,144,119,3.0,143,141
4521,138,288,5.0,137,283
...,...,...,...,...,...
5734,173,402,5.0,172,175
5191,157,85,4.0,156,33
5390,163,18,3.0,162,52
860,28,416,4.0,27,187


In [26]:
x = data_collaborative[['user', 'resto']].values


y = data_collaborative['Place_Ratings'].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values

#  80% data train and 20% data validasion
train_indices = int(0.8 * data_collaborative.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:]
)

print(x, y)

[[187 262]
 [141  83]
 [ 53  10]
 ...
 [162  52]
 [ 27 187]
 [218   6]] [0.5  0.25 0.5  ... 0.5  0.75 0.25]


In [27]:
class RecommenderNet(tf.keras.Model):


  def __init__(self, num_users, num_resto, embedding_size, **kwargs):
    super(RecommenderNet, self).__init__(**kwargs)
    self.num_users = num_users
    self.num_resto = num_resto
    self.embedding_size = embedding_size
    self.user_embedding = layers.Embedding( # layer embedding user
        num_users,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(1e-6)
    )
    self.user_bias = layers.Embedding(num_users, 1) # layer embedding user bias
    self.resto_embedding = layers.Embedding( # layer embeddings
        num_resto,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(1e-6)
    )
    self.resto_bias = layers.Embedding(num_resto, 1) # layer embedding  bias

  def call(self, inputs):
    user_vector = self.user_embedding(inputs[:,0]) #  layer embedding 1
    user_bias = self.user_bias(inputs[:, 0]) #  layer embedding 2
    resto_vector = self.resto_embedding(inputs[:, 1]) #  layer embedding 3
    resto_bias = self.resto_bias(inputs[:, 1]) #  layer embedding 4

    dot_user_resto = tf.tensordot(user_vector, resto_vector, 2)

    x = dot_user_resto + user_bias + resto_bias

    return tf.nn.sigmoid(x) # activation sigmoid

In [28]:
model = RecommenderNet(num_users, num_resto, 50) # intialisation model

# model compile
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [29]:

history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 8,
    epochs = 100,
    validation_data = (x_val, y_val)
)

Epoch 1/100
1000/1000 [==============================] - 3s 2ms/step - loss: 0.6941 - root_mean_squared_error: 0.3460 - val_loss: 0.6934 - val_root_mean_squared_error: 0.3451
Epoch 2/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6876 - root_mean_squared_error: 0.3413 - val_loss: 0.6938 - val_root_mean_squared_error: 0.3454
Epoch 3/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6810 - root_mean_squared_error: 0.3364 - val_loss: 0.6943 - val_root_mean_squared_error: 0.3457
Epoch 4/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6771 - root_mean_squared_error: 0.3336 - val_loss: 0.6950 - val_root_mean_squared_error: 0.3462
Epoch 5/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6711 - root_mean_squared_error: 0.3292 - val_loss: 0.6959 - val_root_mean_squared_error: 0.3468
Epoch 6/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6682 - root_mean_squared_error: 0.3271 - val_l

In [30]:
resto_df = tourism_with_id
df = tourism_rating


user_id = df.User_Id.sample(1).iloc[0]
resto_visited_by_user = df[df.User_Id == user_id]

In [31]:
resto_not_visited = resto_df[~resto_df['Place_Id'].isin(resto_visited_by_user.Place_Id.values)]['Place_Id']
resto_not_visited = list(
    set(resto_not_visited)
    .intersection(set(resto_to_resto_encoded.keys()))
)

resto_not_visited = [[resto_to_resto_encoded.get(x)] for x in resto_not_visited]
user_encoder = user_to_user_encoded.get(user_id)
user_resto_array = np.hstack(
    ([[user_encoder]] * len(resto_not_visited), resto_not_visited)
)

In [32]:
ratings = model.predict(user_resto_array).flatten()

13/13 [==============================] - 0s 1ms/step


In [33]:
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_resto_ids = [
    resto_encoded_to_resto.get(resto_not_visited[x][0]) for x in top_ratings_indices
]

In [34]:
print('Showing recommendations for users: {}'.format(user_id))
print('===' * 9)
print('Resto with high ratings from user')
print('----' * 8)

top_resto_user = (
    resto_visited_by_user.sort_values(
        by = 'Place_Ratings',
        ascending=False
    )
    .head(5)
    .Place_Id.values
)

resto_df_rows = resto_df[resto_df['Place_Id'].isin(top_resto_user)]
for row in resto_df_rows.itertuples():
    print(row.Place_Name)

print('----' * 8)
print('Top 10 resto recommendation')
print('----' * 8)

recommended_resto = resto_df[resto_df['Place_Id'].isin(recommended_resto_ids)]
for row in recommended_resto.itertuples():
    print(row.Place_Name)

Showing recommendations for users: 262
Resto with high ratings from user
--------------------------------
Ocean Ecopark
Pulau Bidadari
Bukit Panguk Kediwung
Kampung Batu Malakasari
Curug Tilu Leuwi Opat
--------------------------------
Top 10 resto recommendation
--------------------------------
Museum Gunung Merapi
Goa Pindul
Jogja Bay Pirates Adventure Waterpark
Pantai Wediombo
Desa Wisata Pulesari
GPIB Immanuel Semarang (Gereja Blenduk)
Taman Srigunting
Water Blaster Bukit Candi Golf
Wisata Lereng Kelir
Kampoeng Kopi Banaran


In [35]:
recommend_by_content_based_filtering('Pantai Watu Kodok')

[['Pantai Kukup', 0.5659440531411729],
 ['Pantai Timang', 0.5185433503874808],
 ['Pantai Drini', 0.5034916444153296],
 ['Pantai Ngandong', 0.48939744234953786],
 ['Pantai Sundak', 0.4855410205632306],
 ['Pantai Ngrawe (Mesra)', 0.4655150705971857],
 ['Pantai Sadranan', 0.44906283441167205],
 ['Pantai Jungwok', 0.42546650148837023],
 ['Pantai Cipta', 0.4101023347137378]]